<img src="https://raw.githubusercontent.com/OpenEnergyPlatform/academy/develop/docs/data/img/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="75" width="75" align="left"/>

# API Tutorial 02 - Upload data using the API

## Learnings
This tutorials will enable you to:

* [Create a table on the OEP using the API](#create-table-using-the-api)
* [Upload data to that table using the API](#upload-data-using-the-api)
* [Upload metadata accompanying the data using the API](#upload-metadata-using-the-api)
* [Delete the table using the API](#delete-table-using-the-api)

## Requirements
To execute the following code you will need to have the following installed:
* Python

* Python packages:
    * `requests`
    * `pandas`
    * jupyter notebook, e.g. `notebook` or `jupyterlab`
* Get your OEP API token from the settings tab in your profile page    
* Skills:
    * use python with jupyter notebooks


## Setup

In [1]:
# install required packages with: pip install requests pandas
# import required packages
import json
import requests as req
import pandas as pd
from random import randint
from getpass import getpass
from os import environ

topic = 'sandbox'
table = f'test_table_{randint(0, 100000)}'
token = environ.get("OEP_API_TOKEN") or getpass('Enter your OEP API token:')

# for read/write, we need to add authorization header
auth_headers = {'Authorization': 'Token %s' % token}
table_api_url = f"https://openenergy-platform.org/api/v0/schema/{topic}/tables/{table}/"

## Create table using the API

requires execution of [setup](#setup)

In [2]:
# TODO: explain / link to data types
table_schema = {
    "columns": [
        # NOTE: first column should be numerical column named `id` .
        # Use `bigserial` if you want the database to create the re
        {"name": "id", "data_type": "bigserial", "primary_key": True},
        {"name": "name", "data_type": "varchar(18)", "is_nullable": False},
        {"name": "is_active", "data_type": "boolean"},
        {"name": "capacity_mw", "data_type": "float"},
        {"name": "installation_datetime_utc", "data_type": "datetime"},        
        {"name": "location", "data_type": "geometry"},
    ]
}

In [3]:
# Creating table needs authentication headers
# Creating table needs table schema json data
res = req.put(table_api_url, json={"query": table_schema}, headers=auth_headers)

# raise Exception if request fails
if not res.ok:
    raise Exception(res.text)

## Upload data using the API

requires execution of [setup](#setup) and already created table

In [4]:
# TODO: explain required data structure

# get example data (from csv)
df = pd.read_csv("https://raw.githubusercontent.com/OpenEnergyPlatform/academy/production/docs/data/upload_tutorial_example_data.csv")

# optionally: clean up nan values
df = df.astype(object).where(pd.notnull(df), None)
data = df.to_dict(orient="records")

# show results in notebook
print(json.dumps(data, indent=4))

[
    {
        "name": "unit1",
        "is_active": true,
        "capacity_mw": 1.2,
        "installation_datetime_utc": "2010-02-03 00:00:00",
        "location": "POINT(53.12 8.345)"
    },
    {
        "name": "unit2",
        "is_active": false,
        "capacity_mw": 2.1,
        "installation_datetime_utc": "2010-01-08",
        "location": null
    },
    {
        "name": "unit3",
        "is_active": true,
        "capacity_mw": 100.0,
        "installation_datetime_utc": "2010-01-02 10:30:00",
        "location": "Point(55.34 7.34)"
    }
]


In [5]:
# Upload data needs authentication headers
# Upload data needs data records in json query
res = req.post(table_api_url + "rows/new", json={"query": data}, headers=auth_headers)

# raise Exception if request fails
if not res.ok:
    raise Exception(res.text)

# TODO: maybe batches, append, not overwrite existing

## Upload metadata using the API

requires execution of [setup](#setup) and already created table

In [6]:
# get metadata (from example file)
metadata = req.get("https://raw.githubusercontent.com/OpenEnergyPlatform/academy/production/docs/data/upload_tutorial_example_data.metadata.json").json()

# show results in notebook
print(json.dumps(metadata, indent=4))

JSONDecodeError: Extra data: line 1 column 4 (char 3)

In [ ]:
# Upload metadata needs authentication headers
# Upload metadata needs metadata json
res = req.post(table_api_url + "meta/", json=metadata, headers=auth_headers)

# raise Exception if request fails
if not res.ok:
    raise Exception(res.text)

## Delete table using the API

requires execution of [setup](#setup) and already created table

In [ ]:
# Deleting tables needs authentication headers
res = req.delete(table_api_url, headers=auth_headers)

# raise Exception if request fails
if not res.ok:
    raise Exception(res.text)